# 엄선된 feature만 사용해서 돌아가는 RF 모델

In [1]:
import numpy as np
import sklearn
from sklearn.ensemble import RandomForestClassifier
from preprocess import merge_main_npy
import pickle

In [2]:
base_dir = "/home/hoseung/Work/data/BBS/"

action = 13
main_list = f"whoismain/{action}/main_list_{action}.txt"
npy_list = merge_main_npy(base_dir+main_list, prefix=base_dir + "npy_a/")

# 스켈레톤이 아주 stochastic하기 때문에 5장 중 median으로 smoothing

In [56]:
def smoothed_frame_sample(scene, fps = 1, window_size = 5):
    """일정 시간에 한번씩 ... 영상 길이가 다를 경우에 뒤를 0으로 채워야함 
    """
    FPS_ORIGINAL=10 # 인 것 같음...
    
    nskip = int(FPS_ORIGINAL/fps)
    nframe = np.ceil((len(scene) - window_size)/ nskip).astype(int)

    sub = np.zeros(nframe, dtype=scene.dtype) #scene.dtype - frame

    for i in range(nframe):
        temp = scene[i*nskip:i*nskip+window_size]

        for feat in temp.dtype.names: # recarry라서 한 번에 np.mean 불가능
            sub[i][feat] = np.median(temp[feat])

    return sub


def smoothed_frame_N(scene, nframe = 10, window_size = 5, shift=0):
    FPS_ORIGINAL=10 # 인 것 같음...
    
    nskip = int((len(scene)-shift) / nframe)
    #nframe = np.ceil((len(scene) - window_size)/ nskip).astype(int)

    sub = np.zeros(nframe, dtype=scene.dtype) #scene.dtype - frame

    for i in range(nframe):
        temp = scene[i*nskip+shift:i*nskip+window_size+shift]

        for feat in temp.dtype.names: # recarry라서 한 번에 np.mean 불가능
            sub[i][feat] = np.median(temp[feat])

    return sub


def ravel_rec(sub, return_feature=False, fields_to_remove=[]):
    """remove pre-defined joints and flatten the remaing skeletons. 
    """
    #print("a", fields_to_remove)
    #print("b", np.unique(['frame'] + fields_to_remove))
    fields_to_remove = list(np.unique(['frame'] + fields_to_remove))
    #print("c", fields_to_remove)
    
    vec=[]
    if return_feature:
        features=[]
        for i, line in enumerate(sub):
            this_line=[]
            for ff in line.dtype.names:
                ff_frame = ff+f"_{line['frame']}"
                print(ff)
                print(fields_to_remove)
                if ff not in fields_to_remove and ff_frame not in fields_to_remove:
                    this_line.append(line[ff])
                    features.append(ff_frame)
                    print("len(this_line)", len(this_line))
            vec.extend(this_line)
                    
        return features, np.array(vec)
    else:
        for line in sub:
            this_line=[]
            for ff in line.dtype.names:
                ff_frame = ff+f"_{line['frame']}"
                if ff not in fields_to_remove and ff_frame not in fields_to_remove:
                    this_line.append(line[ff])
            vec.extend(this_line)
            #vec.extend([line[ff] for ff in line.dtype.names if ff not in fields_to_remove])

    #print("LAST", fields_to_remove)
    return np.array(vec)

In [68]:
def has_match(longer_string, ss_list):
    """Check if a string in a list of strings is a part of the given string
    
    example
    -------
    has_match('ab', ['abnormal', 'cdbg']) 
    > True
    """
    return bool(sum([target in ss for target in ss_list]))

In [60]:
ss_list = ['abnormal', 'cdbg']
ss = 'ab'
sum([(target in ss for target in ss_list])

0

In [66]:
sum(tf)

1

In [67]:
has_match('abnormal', ['ab', 'cdbg']) 

True

In [57]:
# 13번, 
fields_to_remove =["lhand", 'lwrist', 'lshoulder'] 

scene = np.load(npy_list[0]['npy'])
nframe = 2
frame_skip = 2
data = []
label=[]
feature=[]

feature_ready=False
for i, fn_npy in enumerate(npy_list[:1]):
    
    if fn_npy['main'] in [0,1]:
        
        #try:
        if True:
            scene = np.load(fn_npy['npy'])
            for j in range(frame_skip):
                sub = smoothed_frame_N(scene, nframe=nframe, shift=j)
            
                # feature 기록
                if i == j == 0:
                    feature, rav_sub = ravel_rec(sub, 
                                                 return_feature=True,
                                                 fields_to_remove=fields_to_remove)
                    #print(fields_to_remove)
                else:
                    rav_sub = ravel_rec(sub, fields_to_remove=fields_to_remove)
                data.append(rav_sub)
                label.append(fn_npy['score'])
                #print("?",fields_to_remove)
        #except:
            print("Main person was guessed, but not actually detected... ", fn_npy['npy'])


frame
['frame', 'lhand', 'lshoulder', 'lwrist']
xl_hand
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 1
yl_hand
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 2
xl_elbow
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 3
yl_elbow
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 4
xl_shoulder
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 5
yl_shoulder
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 6
xr_shoulder
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 7
yr_shoulder
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 8
xr_elbow
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 9
yr_elbow
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 10
xr_hand
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 11
yr_hand
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 12
xhead
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_line) 13
yhead
['frame', 'lhand', 'lshoulder', 'lwrist']
len(this_li

In [49]:
data[0].dtype

dtype('float64')

In [50]:
data[0].shape

(180,)

In [72]:
get_feature_every_frame(scene, to_remove=['hand', 'knee', 'nose'])

['frame',
 'xl_hand',
 'yl_hand',
 'xl_elbow',
 'yl_elbow',
 'xl_shoulder',
 'yl_shoulder',
 'xr_shoulder',
 'yr_shoulder',
 'xr_elbow',
 'yr_elbow',
 'xr_hand',
 'yr_hand',
 'xhead',
 'yhead',
 'xneck',
 'yneck',
 'xpelvis',
 'ypelvis',
 'xl_foot',
 'yl_foot',
 'xl_knee',
 'yl_knee',
 'xl_hip',
 'yl_hip',
 'xr_hip',
 'yr_hip',
 'xr_knee',
 'yr_knee',
 'xr_foot',
 'yr_foot']

In [73]:
has_match("xl_hand", ['hand', 'knee', 'nose'])

False

In [71]:
def get_feature_every_frame(arr, to_remove=[""]):
    """make list of field names to keep.
    
    example
    -------
    if to_remove = ['hand'], all fields containing 'hand' are removed
    """
    to_keep = []
    for name in arr.dtype.names:
        #for target in to_remove:
        #if sum([target in name for target in to_remove]) == 0:
        if not has_match(name, to_remove):
            to_keep.append(name)

    return to_keep


def remove_fields(arr, fields_to_remove):
    names_to_keep = [name for name in arr.dtype.names if name not in fields_to_remove]
    return arr[names_to_keep]

In [105]:
data = np.stack(data)
label = np.array(label)
print(data.shape)

(8180, 180)


In [106]:
from sklearn import preprocessing

sc = preprocessing.MinMaxScaler(feature_range=[0,1])

data = sc.fit_transform(data)

In [107]:
X_train, X_test, Y_train, Y_test = sklearn.model_selection.train_test_split(data, 
                                                                            label, 
                                                                            test_size=0.7,
                                                                            stratify=label)

In [120]:
small = True

if small:
    ntree = 20
    max_depth = 6
    fn_model_out = "./trained_model13_s.pickle"
    fn_data_out = "BBS_dataset_s.pickle"
else:
    ntree = 100
    max_depth = 7
    fn_model_out = "./trained_model13.pickle"
    fn_data_out = "BBS_dataset.pickle"

    
if True:
    pickle.dump({'train_x':X_train,
                 'train_y':Y_train,
                 'valid_x':X_test,
                 'valid_y':Y_test,
                 'label' : label}, open(fn_data_out, "wb"))
else:
    dataset = pickle.load(open("BBS_dataset.pickle", "rb"))
    X_train = dataset['train_x']
    Y_train = dataset['train_y']
    X_test = dataset['valid_x']
    Y_test = dataset['valid_y']
    
    
model = RandomForestClassifier(n_estimators=ntree, max_depth=max_depth)

model.fit(X_train, Y_train)

from sklearn.metrics import accuracy_score
pred = model.predict(X_test)
print("정확도 :{0:.3f}".format(accuracy_score(Y_test, pred)))

pickle.dump(model, open(fn_model_out, "wb"))

정확도 :0.901


In [15]:
trees = model.estimators_

dt = trees[0]

# Feature trackback

In [17]:
farr = dt.tree_.feature[dt.tree_.feature > 0]

In [20]:
features = [ff+f"_{i}" for i in range(nframe) for ff in sub.dtype.names if ff not in "frame" ]
features = np.array(features)

In [21]:
features[farr]

array(['yhead_3', 'xr_elbow_9', 'xr_elbow_2', 'yr_foot_3',
       'yl_shoulder_4', 'xneck_7', 'xl_shoulder_0', 'xr_shoulder_1',
       'xr_shoulder_4', 'yl_elbow_6', 'yl_foot_1', 'xr_foot_3',
       'yr_shoulder_9', 'xl_hip_6', 'xhead_5', 'xl_hip_0', 'yl_foot_6',
       'xr_elbow_6', 'ypelvis_7', 'xl_elbow_1', 'yr_foot_7', 'xl_foot_9',
       'xl_foot_2', 'xhead_2', 'xr_foot_4', 'yr_elbow_4', 'xr_knee_4',
       'xr_knee_5', 'yhead_6', 'xl_hand_6', 'xl_hip_2', 'yr_hand_3',
       'xneck_0', 'xl_shoulder_4', 'yr_elbow_7', 'yl_foot_0',
       'xr_shoulder_3', 'xl_foot_5', 'xr_hand_2', 'yl_elbow_5',
       'yr_elbow_2', 'xr_foot_2', 'yl_knee_3', 'xl_elbow_4', 'xpelvis_0',
       'yr_elbow_3', 'yl_foot_1', 'yl_knee_9', 'yhead_0', 'yr_hand_9',
       'yl_knee_5', 'xl_shoulder_6', 'yl_shoulder_6', 'xl_foot_1',
       'yl_elbow_2', 'yhead_0', 'xr_elbow_6'], dtype='<U13')

# Ablation test

# Test with new data

In [208]:
import kinect_utils as ku

import pickle
point_pickle = "/home/hoseung/Work/Kinect/pykinect_BBS/G1/000/BT/bodytracking_data.pickle"
klist = pickle.load(open(point_pickle, "rb"))

scene = ku.kinect2mobile_direct(klist)

nframe = 10 
shift=0
sub = smoothed_frame_N(scene, nframe=nframe, shift=j)
rav_sub = ravel_rec(sub)[np.newaxis, :]
pickle.dump(rav_sub, open("rav_example.pickle", "wb"))

In [206]:
pwd

'/home/hoseung/Work/fhenrf/pose'

In [193]:
model.predict(rav_sub[np.newaxis,:])

array([2])